In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import json
import numpy as np
import spacy
import itertools
import json
import math
import os
import random
from collections import defaultdict
from collections.abc import Sized
from itertools import chain, repeat
from pathlib import Path
from typing import (
    TYPE_CHECKING,
    List,
    Optional,
    Tuple,
)

import spacy
import torch
from accelerate import Accelerator
from confit import Config
from confit import Cli
from confit.utils.random import set_seed
from rich_logger import RichTablePrinter
from torch.utils.data import DataLoader
from tqdm import tqdm

from eds_pseudo.adapter import PseudoReader
from eds_pseudo.scorer import PseudoScorer
from edsnlp.core.pipeline import Pipeline
from edsnlp.core.registries import registry
from edsnlp.optimization import LinearSchedule, ScheduledOptimizer
from edsnlp.pipes.trainable.embeddings.transformer.transformer import Transformer
from edsnlp.utils.collections import batchify
from edsnlp.utils.typing import AsList

BASE_DIR = Path.cwd()

import sys
from pathlib import Path
parent_dir = str(Path.cwd().parent)
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from scripts.train import BatchSizeArg, LengthSortedBatchSampler, SubBatchCollater
import edsnlp


Le but de ce notebook est de montrer comment passer du fichier train.py et de la config de edsnlp a un notebook qui permet de calculer le premier loss sur le premier batch

# 1 - Chargement des configs et du pipeline

In [17]:
data_seed: int = 42
max_steps: int = 1000
batch_size: BatchSizeArg = (10,"words")
embedding_lr: float = 5e-5
task_lr: float = 3e-4
validation_interval: int = 10
grad_max_norm: float = 5.0
grad_accumulation_max_tokens: int = 96 * 128
scorer: PseudoScorer
output_dir: Optional[Path] = None
cpu: bool = True

#chargement de la config, soit avce resolve soit sans
cfg = Config.from_disk("../configs/config.cfg")
cfg

{'vars': {'ml_spans': 'pseudo-ml',
  'rb_spans': 'pseudo-rb',
  'hybrid_spans': 'ents',
  'limit': -1,
  'labels': ['ADRESSE',
   'DATE',
   'DATE_NAISSANCE',
   'IPP',
   'MAIL',
   'NDA',
   'NOM',
   'PRENOM',
   'SECU',
   'TEL',
   'VILLE',
   'ZIP']},
 'nlp': {'@core': 'pipeline',
  'lang': 'eds',
  'pipeline': ['normalizer',
   'simple-rules',
   'addresses',
   'context',
   'ner',
   'clean',
   'merge',
   'dates-normalizer'],
  'batch_size': 32,
  'components': ${components}},
 'components': {'normalizer': {'@factory': 'eds.normalizer'},
  'sentencizer': {'@factory': 'eds.sentences'},
  'remove-lowercase': {'@factory': 'eds.remove_lowercase'},
  'dates': {'@factory': 'eds_pseudo.dates', 'span_setter': ${vars.rb_spans}},
  'simple-rules': {'@factory': 'eds_pseudo.simple_rules',
   'pattern_keys': ['TEL', 'MAIL', 'SECU'],
   'span_setter': ${vars.rb_spans}},
  'addresses': {'@factory': 'eds_pseudo.addresses',
   'span_setter': ${vars.rb_spans}},
  'context': {'@factory': 'eds_

In [14]:
#dans ce cas, la configuration est chargée grâce au module config et à tous les décorateurs qui permettent d'enregistrer les fonction dans le registre
#par exemple, grâce à ces indications dans la config
#[training_docs.source]
#@readers = "json"  # or parquet or standoff
#converter = "pseudo"  # defined in eds_pseudo/adapter.py
#
#confit a trouvé la classe qui permettait de lire la donnée, elel se trove dans le fichier adapater.py
#et s'identifie grace à:
#@registry.factory.register("eds.pseudo_dict2doc", spacy_compatible=False)
#class PseudoDict2DocConverter:
#
#le résultat se voit dans la cfg_resolved
#'training_docs': {...
#  'source': Stream(
#    reader=JsonReader(path='/export/home/amessager/projet/pseudo_champs_courts/data/training/private_small_train_data.jsonl', shuffle=False, loop=False),
#    ops=[
#      map(<eds_pseudo.adapter.PseudoDict2DocConverter object at 0x7f4024eeafc0>)
#    ],
cfg_resolved = Config.from_disk("../configs/config.cfg").resolve(registry=edsnlp.registry)
cfg_resolved

2025-09-29 08:48:20.903 | INFO     | edsnlp.data.json:__init__:65 - Found 1 file under /export/home/amessager/projet/pseudo_champs_courts/src/eds_pseudo/demo/public_small_train_data.jsonl
2025-09-29 08:48:20.905 | INFO     | edsnlp.data.json:__init__:65 - Found 1 file under /export/home/amessager/projet/pseudo_champs_courts/src/eds_pseudo/demo/public_small_dev_data.jsonl
2025-09-29 08:48:20.906 | INFO     | edsnlp.data.json:__init__:65 - Found 1 file under /export/home/amessager/projet/pseudo_champs_courts/src/eds_pseudo/demo/public_small_test_data.jsonl


{'vars': {'ml_spans': 'pseudo-ml',
  'rb_spans': 'pseudo-rb',
  'hybrid_spans': 'ents',
  'limit': -1,
  'labels': ['ADRESSE',
   'DATE',
   'DATE_NAISSANCE',
   'IPP',
   'MAIL',
   'NDA',
   'NOM',
   'PRENOM',
   'SECU',
   'TEL',
   'VILLE',
   'ZIP']},
 'nlp': Pipeline(lang=eds, pipes={
   "normalizer": eds.normalizer,
   "simple-rules": eds_pseudo.simple_rules,
   "addresses": eds_pseudo.addresses,
   "context": eds_pseudo.context,
   "ner": eds.ner_crf,
   "clean": eds_pseudo.clean,
   "merge": eds_pseudo.merge,
   "dates-normalizer": eds_pseudo.dates_normalizer
 }),
 'components': {'normalizer': Draft[create_component],
  'sentencizer': Draft[SentenceSegmenter],
  'remove-lowercase': Draft[create_component],
  'dates': Draft[PseudonymisationDates],
  'simple-rules': Draft[Pseudonymisation],
  'addresses': Draft[PseudonymisationAddresses],
  'context': Draft[ContextMatcher],
  'embedding': TextCnnEncoder(
    (embedding): Transformer(
      (transformer): CamembertModel(
       

In [20]:
nlp=edsnlp.load(cfg_resolved)

# 2 - Chargement des données

In [21]:
#exemple de fichier:
#{"note_id":"synthetic_11831851499","note_text":"krier franck rivaill59be@clinique-lyon.fr","entities":[{"start":0,"end":5,"label":"NOM"},{"start":6,"end":12,"label":"PRENOM"},{"start":13,"end":41,"label":"MAIL"}]}
raw_val_docs = edsnlp.data.read_json(
    cfg['val_docs']['source']['path'],#le chemin vers le fichier source, un jsonl, 
    converter="eds.pseudo_dict2doc",#dans le fichier adapater.py, convertit en format doc le json, c.f. commentaire config pour plus d'infos
    span_setter="pseudo-ml",#très important, dans le fichier config, les entités à identifier doivent appartenir au span "pseudo-ml", sinon le preprocessing ne produira pas de targets
)
augmented_val_docs = PseudoReader(raw_val_docs)(nlp)

raw_train_docs = edsnlp.data.read_json(
    cfg['training_docs']['source']['path'],
    converter="eds.pseudo_dict2doc",
    span_setter="pseudo-ml",
)
augmented_train_docs = PseudoReader(raw_train_docs,max_length=15)(nlp)#la max length dans la config est a 256, mais mis a 15 pour faciliter la compréhension

2025-09-29 08:49:11.521 | INFO     | edsnlp.data.json:__init__:65 - Found 1 file under /export/home/amessager/projet/pseudo_champs_courts/src/eds_pseudo/demo/public_small_dev_data.jsonl
2025-09-29 08:49:11.523 | INFO     | edsnlp.data.json:__init__:65 - Found 1 file under /export/home/amessager/projet/pseudo_champs_courts/src/eds_pseudo/demo/public_small_train_data.jsonl


In [22]:
#illustration du span setter:
raw_train_docs = edsnlp.data.read_json(
    cfg['training_docs']['source']['path'],#le chemin vers le fichier source, un jsonl, 
    converter="eds.pseudo_dict2doc",#dans le fichier adapater.py, convertit en format doc le json, c.f. commentaire config pour plus d'infos
    span_setter="pseudo-ml",#très important, dans le fichier config, les entités à identifier doivent appartenir au span "pseudo-ml", sinon le preprocessing ne produira pas de targets
)
l=list(raw_train_docs)#stream monté en mémoire
print(l[0])
print("note_id:",l[0]._.note_id)
print("entities",l[0].ents)
print("span:",l[0].spans)
print("comment les entitiés sont stockées dans le span:",l[0].spans['pseudo-ml'][0],"->",l[0].spans['pseudo-ml'][0].label_)

2025-09-29 08:49:12.201 | INFO     | edsnlp.data.json:__init__:65 - Found 1 file under /export/home/amessager/projet/pseudo_champs_courts/src/eds_pseudo/demo/public_small_train_data.jsonl


krier franck rivaill59be@clinique-lyon.fr
note_id: synthetic_11831851499
entities ()
span: {'pseudo-ml': [krier, franck, rivaill59be@clinique-lyon.fr]}
comment les entitiés sont stockées dans le span: krier -> NOM


In [23]:
#illustration du span setter:
raw_train_docs = edsnlp.data.read_json(
    cfg['training_docs']['source']['path'],#le chemin vers le fichier source, un jsonl, 
    converter="eds.pseudo_dict2doc",#dans le fichier adapater.py, convertit en format doc le json, c.f. commentaire config pour plus d'infos
    #span_setter="pseudo-ml",#très important, dans le fichier config, les entités à identifier doivent appartenir au span "pseudo-ml", sinon le preprocessing ne produira pas de targets
)
#dans ces conditions, il n'y aura pas d'apprentissage parce que il n'y a pas de span pseudo-ml
l=list(raw_train_docs)
print(l[0])
print("note_id:",l[0]._.note_id)
print("span:",l[0].spans)
print("entities",l[0].ents)

2025-09-29 08:49:15.235 | INFO     | edsnlp.data.json:__init__:65 - Found 1 file under /export/home/amessager/projet/pseudo_champs_courts/src/eds_pseudo/demo/public_small_train_data.jsonl


krier franck rivaill59be@clinique-lyon.fr
note_id: synthetic_11831851499
span: {'NOM': [krier], 'PRENOM': [franck], 'MAIL': [rivaill59be@clinique-lyon.fr]}
entities (krier, franck, rivaill59be@clinique-lyon.fr)


In [24]:
ner=nlp.pipeline[4]#la phase de NER (CNN(Transformer))
print(ner[0])
print(ner[1].target_span_getter)#c'est ici que l'on voit ou sont définis les spans rercherchés pour les targets

ner
{'pseudo-ml': True}


In [25]:
type(ner[1])

edsnlp.pipes.trainable.ner_crf.ner_crf.TrainableNerCrf

In [26]:
ner[1]#c'est un nn.module

TrainableNerCrf(
  (embedding): TextCnnEncoder(
    (embedding): Transformer(
      (transformer): CamembertModel(
        (embeddings): CamembertEmbeddings(
          (word_embeddings): Embedding(32007, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): CamembertEncoder(
          (layer): ModuleList(
            (0-11): 12 x CamembertLayer(
              (attention): CamembertAttention(
                (self): CamembertSdpaSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                  (dropout): Dropout(p=0.1, inplace=False)
            

In [27]:
#Remarque importante: 
#lorsque le raw_train_docs est défini, c'est un stream, cad qu'il n'est pas chargé avant qu'il soit appelé, un peu comme un iterator
#pour que edsnlp puisse tokenizer les mots, la librairie edsnlp a besoin de définir un vocabulaire, l'ensmeble du vocabualire n'est pas 
#prérempli (il y aurait trop de mo) mais construit au fur et à mesure que les mots sont découverts
#donc si raw_train_docs est chargé en mémoire, avec une list() par exemple, le vocabulaire est construit a partir du corpus, 
#et ne sera pas nécessairement le meme que celui du pipeline nlp
#c'est pratiquement la seule raison pour laquelle le pipeline nlp est appelé lors de construction du texte d'entrainement
raw_train_docs = edsnlp.data.read_json(
    cfg['training_docs']['source']['path'],
    converter="eds.pseudo_dict2doc",
    span_setter="pseudo-ml",
)
augmented_train_docs = PseudoReader(list(raw_train_docs),max_length=15)(nlp)
l=list(augmented_train_docs)#stream chargé en mémoire
l[0].vocab == nlp.vocab

2025-09-29 08:49:20.423 | INFO     | edsnlp.data.json:__init__:65 - Found 1 file under /export/home/amessager/projet/pseudo_champs_courts/src/eds_pseudo/demo/public_small_train_data.jsonl


False

In [28]:
raw_train_docs = edsnlp.data.read_json(
    cfg['training_docs']['source']['path'],
    converter="eds.pseudo_dict2doc",
    span_setter="pseudo-ml",
)
augmented_train_docs = PseudoReader(raw_train_docs,max_length=15)(nlp)
l=list(augmented_train_docs)#stream chargé en mémoire
l[0].vocab == nlp.vocab

2025-09-29 08:49:21.369 | INFO     | edsnlp.data.json:__init__:65 - Found 1 file under /export/home/amessager/projet/pseudo_champs_courts/src/eds_pseudo/demo/public_small_train_data.jsonl


True

In [29]:
#Note: on peut remarquer que le texte produit par le PseudoReader est maintenant composé de "phrase" de taille max 15
#le deuxieme exemple dans le raw_train_doc est beaucoup plus long
print(l[1])

mavoungou bouanga cecile Admission en service de gériatrie Besoin d'aide matérielle urgent


# 3 - Extraction des targets et créations des tenseurs

In [30]:
raw_train_docs = edsnlp.data.read_json(
    cfg['training_docs']['source']['path'],
    converter="eds.pseudo_dict2doc",
    span_setter="pseudo-ml",
)
augmented_train_docs = PseudoReader(raw_train_docs,max_length=15)(nlp)

#nlp.post_init sert à finaliser l'initialisation du pipeline NLP en lui montrant un échantillon des données d'entraînement.
#Cette étape permet aux différents composants du pipeline, comme le vocabulaire et le tokenizer,
#de s'adapter aux données spécifiques qu'ils vont traiter avant que l'entraînement ne commence réellement.
#cela permet nottamment de regarder le nombre de classe à traiter et de définir en conséquence le nombre de têtes classificatrices
#donc la taille du modèle de sortie
nlp.post_init(augmented_train_docs)

preprocessed = list(
    nlp.preprocess_many(augmented_train_docs, supervision=True).set_processing(
        show_progress=True
    )
)

preprocessed[0]

2025-09-29 08:49:25.685 | INFO     | edsnlp.data.json:__init__:65 - Found 1 file under /export/home/amessager/projet/pseudo_champs_courts/src/eds_pseudo/demo/public_small_train_data.jsonl
133it [00:00, 1245.15it/s]


{'ner/embedding/embedding/input_ids': [[1482,
   4420,
   6125,
   496,
   3366,
   1872,
   4303,
   3853,
   1138,
   2266,
   216,
   1660,
   1120,
   26,
   1107,
   88,
   9,
   427]],
 'ner/embedding/embedding/word_tokens': [[0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17]],
 'ner/embedding/embedding/word_lengths': [[2, 2, 3, 1, 1, 1, 3, 1, 2, 1, 1]],
 'ner/embedding/embedding/prompts': [[]],
 'ner/embedding/embedding/stats/tokens': 18,
 'ner/embedding/embedding/stats/words|ner/stats/ner_words': 11,
 'ner/embedding/embedding/stats/contexts': 1,
 'ner/lengths': [11],
 'ner/targets': [[[0, 0, 0, 0, 0, 4, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0, 4, 0, 0, 0],
   [0, 0, 0, 0, 2, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 1, 0, 0, 

In [33]:
raw_train_docs = edsnlp.data.read_json(
    cfg['training_docs']['source']['path'],
    converter="eds.pseudo_dict2doc",
    span_setter="pseudo-ml",
)
doc=list(raw_train_docs)[0]
doc.spans

2025-09-29 08:50:03.851 | INFO     | edsnlp.data.json:__init__:65 - Found 1 file under /export/home/amessager/projet/pseudo_champs_courts/src/eds_pseudo/demo/public_small_train_data.jsonl


{'pseudo-ml': [krier, franck, rivaill59be@clinique-lyon.fr]}

In [34]:
#Il faut remarquer que le preprocessing a créer les targets selon la méthode BIOUL:
#1 signifie inside
#2 signifie beginning
#3 signifie end
#4 signifie entité composée d"un seul token
#la pharse est toujours la meme: krier franck rivaill59be@clinique-lyon.fr
#NOM (token unique=4) PRENOM (token unique=4) MAIL_début (token=2) MAIL_inside (token=1) MAIL_inside (token=1) ... MAIL_inside (token=1) MAIL_fin (token=3)
labels=nlp.pipes.ner.labels
target=preprocessed[0]['ner/targets'][0]
[labels[j]+f'-{target[i][j]}' for i in range(len(target)) for j in range(len(target[0])) if target[i][j]>0]

['NOM-4',
 'PRENOM-4',
 'MAIL-2',
 'MAIL-1',
 'MAIL-1',
 'MAIL-1',
 'MAIL-1',
 'MAIL-1',
 'MAIL-1',
 'MAIL-1',
 'MAIL-3']

In [35]:
#Dans le code c'est fait de la facon suivante
#https://github.com/aphp/edsnlp/blob/c7ae3441d39450749d6225c25596da666a374d6e/edsnlp/pipes/trainable/ner_crf/ner_crf.py#L402
augmented_train_docs = PseudoReader(raw_train_docs,max_length=15)(nlp)
prep={}
supervision=True
for name, component in nlp.pipeline:
    if name not in "disabled":
        prep_comp = (
            component.preprocess_supervised(list(augmented_train_docs)[0])
            if supervision and hasattr(component, "preprocess_supervised")
            else component.preprocess(list(augmented_train_docs)[0])
            if hasattr(component, "preprocess")
            else None
        )
        if prep_comp is not None:
            prep[name] = prep_comp
prep

{'ner': {'embedding': {'embedding': {'input_ids': [[1482,
      4420,
      6125,
      496,
      3366,
      1872,
      4303,
      3853,
      1138,
      2266,
      216,
      1660,
      1120,
      26,
      1107,
      88,
      9,
      427]],
    'word_tokens': [[0,
      1,
      2,
      3,
      4,
      5,
      6,
      7,
      8,
      9,
      10,
      11,
      12,
      13,
      14,
      15,
      16,
      17]],
    'word_lengths': [[2, 2, 3, 1, 1, 1, 3, 1, 2, 1, 1]],
    'prompts': [[]],
    'stats': {'tokens': 18, 'words': 11, 'contexts': 1}}},
  'lengths': [11],
  '$contexts': [krier franck rivaill59be@clinique-lyon.fr],
  'stats': {'ner_words': 11},
  'targets': [[[0, 0, 0, 0, 0, 4, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 4, 0, 0, 0],
    [0, 0, 0, 0, 2, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 

# 4 creation des batchs

In [36]:
#le transformer a proprement parlé
trf_pipe = next(
        module
        for name, pipe in nlp.torch_components()
        for module_name, module in pipe.named_component_modules()
        if isinstance(module, Transformer)
    )
trf_pipe

Transformer(
  (transformer): CamembertModel(
    (embeddings): CamembertEmbeddings(
      (word_embeddings): Embedding(32007, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): CamembertEncoder(
      (layer): ModuleList(
        (0-11): 12 x CamembertLayer(
          (attention): CamembertAttention(
            (self): CamembertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): CamembertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
            

In [37]:
print(batch_size)

(10, 'words')


In [38]:
#LengthSortedBatchSampler
#Son but est d'accélérer l'entraînement en minimisant le "padding" (bourrage).
#Pour cela, il trie tous les documents par longueur avant de les grouper.
#Il crée ainsi des lots (batchs) contenant des textes de tailles similaires.
#Cela rend le traitement par les modèles de type transformers plus efficace.
#Enfin, il mélange légèrement les lots pour conserver de l'aléa dans l'entraînement.
#
#SubBatchCollater
#Son rôle est d'éviter les erreurs de mémoire sur le GPU ("out of memory").
#Il prend un lot (batch) et le divise en "mini-lots" si nécessaire.
#La division se produit si le nombre total de tokens du lot dépasse un seuil.
#Chaque mini-lot est ainsi garanti de ne pas surcharger la mémoire.
#Il formate ensuite chaque mini-lot pour qu'il soit prêt pour le modèle.

#les deux classes sont dans le fichier train.py pour plus d'infos
dataloader = DataLoader(
        preprocessed,
        batch_sampler=LengthSortedBatchSampler(
            preprocessed,
            batch_size=batch_size[0],
            batch_unit=batch_size[1],
        ),
        collate_fn=SubBatchCollater(
            nlp,
            trf_pipe,
            grad_accumulation_max_tokens=grad_accumulation_max_tokens,
        ),
    )


In [39]:
batch=next(iter(dataloader))
batch

[{'ner': {'embedding': {'embedding': {'input_ids': FoldedTensor([[    5,  4844,  1699,    35,    43,   277, 20385,     7,   470,
                       26,  6859,    21,     7,  4037,   134,     7, 17016,    15,
                       13,  5619,  8597,    62,  2141,     6]]),
     'word_offsets': FoldedTensor([ 0,  3,  4,  6,  7,  8,  9, 10, 12, 13, 14, 15, 16, 17, 18]),
     'word_indices': tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
             19, 20, 21, 22]),
     'empty_word_indices': tensor([], dtype=torch.int64),
     'stats': {'tokens': 22, 'words': 15, 'contexts': 1},
     '__cache_key__': ('collate',
      'Transformer<139658460562256>',
      -7431332083199045598)},
    '__cache_key__': ('collate',
     'TextCnnEncoder<139657777112384>',
     -2647470331429679846)},
   'stats': {'ner_words': 15},
   'targets': tensor([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 4, 0, 0, 0, 0],


# 5 - optimiseur

In [40]:
trained_pipes = nlp.torch_components()
print("Training", ", ".join([name for name, c in trained_pipes]))

trf_params = set(trf_pipe.parameters())
params = set(nlp.parameters())
optimizer = ScheduledOptimizer(
    torch.optim.AdamW(
        [
            {
                "params": list(params - trf_params),
                "lr": task_lr,
                "schedules": [
                    LinearSchedule(
                        total_steps=max_steps,
                        warmup_rate=0.1,
                        start_value=task_lr,
                        path="lr",
                    )
                ],
            },
            {
                "params": list(trf_params),
                "lr": embedding_lr,
                "schedules": [
                    LinearSchedule(
                        total_steps=max_steps,
                        warmup_rate=0.1,
                        start_value=0,
                        path="lr",
                    )
                ],
            },
        ]
    )
)
grad_params = {p for group in optimizer.param_groups for p in group["params"]}
print(
    "Optimizing:"
    + "".join(
        f"\n - {len(group['params'])} params "
        f"({sum(p.numel() for p in group['params'])} total)"
        for group in optimizer.param_groups
    )
)
print(f"Not optimizing {len(params - grad_params)} params")

Training ner
Optimizing:
 - 6 params (2399282 total)
 - 200 params (110624256 total)
Not optimizing 0 params


In [41]:
accelerator = Accelerator(cpu=cpu)
trained_pipes = dict(nlp.torch_components())
print("Device:", accelerator.device)
[dataloader, optimizer, *accelerated_pipes] = accelerator.prepare(
    dataloader,
    optimizer,
    *trained_pipes.values(),
)
trained_pipes = list(zip(trained_pipes.keys(), accelerated_pipes))
del accelerated_pipes

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Device: cpu


# 6 - calculs des metriques

In [42]:
from confit.utils.random import set_seed

set_seed(42)

In [43]:
scorer=PseudoScorer(**cfg_resolved['scorers'])

In [44]:
#la méthode par défaut pour mesurer la précision et la sensitivé est la méthode hybride
#il y a originellement deux méthodes: une basées uniquement sur le transformer et l'autre basée sur des règles
#la méthode hybride est une hybridation de ces deux méthodes
#le scorer utilise cette méthode et par défaut le span utilisé est ents
#il faut donc recharger les données avec aussi le span ents
scorer.hybrid_spans

'ents'

In [45]:
raw_train_docs = edsnlp.data.read_json(
    cfg['training_docs']['source']['path'],
    converter="eds.pseudo_dict2doc",
    span_setter=["pseudo-ml", "pseudo-rb", "ents"],
)
augmented_train_docs = PseudoReader(raw_train_docs,max_length=15)(nlp)
docs=list(augmented_train_docs)
doc=docs[0]
docs=[doc]#simplification a 1 doc
doc

2025-09-29 08:50:21.547 | INFO     | edsnlp.data.json:__init__:65 - Found 1 file under /export/home/amessager/projet/pseudo_champs_courts/src/eds_pseudo/demo/public_small_train_data.jsonl


krier franck rivaill59be@clinique-lyon.fr

In [46]:
#On voit bien que les entités sont dans les ents et dans les spans pseudo-ml et pseudo-rb
print(doc.ents)
print(doc.spans)

(krier, franck, rivaill59be@clinique-lyon.fr)
{'pseudo-ml': [krier, franck, rivaill59be@clinique-lyon.fr], 'pseudo-rb': [krier, franck, rivaill59be@clinique-lyon.fr], 'pollutions': [rivaill59be@clinique-lyon.fr]}


In [47]:
from edsnlp.scorers import make_examples
#on nettoie pour faire la prédiction et ne pas tacher le processus
clean_docs: List[spacy.tokens.Doc] = [d.copy() for d in [doc]]
for d in clean_docs:
    d.ents = []
    d.spans.clear()
preds=list(nlp.pipe(clean_docs).set_processing(show_progress=True))
#Les objets examples sont la structure de données standard pour évaluer un modèle.
#Chaque Example est une paire qui apparie la vérité terrain (le document original doc avec les bonnes annotations) avec la prédiction du modèle (preds).
#Cette mise en correspondance est essentielle pour les fonctions d'évaluation (scorers).
#Elles peuvent ainsi facilement confronter la "bonne réponse" et la "copie de l'élève".
#Cela leur permet de calculer des métriques comme la précision, le rappel et le F1-score.
examples = make_examples([doc], preds)
examples

1it [00:01,  1.67s/it]


[{'doc_annotation': {'cats': {}, 'entities': ['U-NOM', 'U-PRENOM', 'B-MAIL', 'I-MAIL', 'I-MAIL', 'I-MAIL', 'I-MAIL', 'I-MAIL', 'I-MAIL', 'I-MAIL', 'L-MAIL'], 'spans': {'pseudo-ml': [(0, 5, 'NOM', ''), (6, 12, 'PRENOM', ''), (13, 41, 'MAIL', '')], 'pseudo-rb': [(0, 5, 'NOM', ''), (6, 12, 'PRENOM', ''), (13, 41, 'MAIL', '')], 'pollutions': [(13, 41, 'web', '')]}, 'links': {}}, 'token_annotation': {'ORTH': ['krier', 'franck', 'rivaill', '59', 'be', '@', 'clinique', '-', 'lyon', '.', 'fr'], 'SPACY': [True, True, False, False, False, False, False, False, False, False, False], 'TAG': ['', '', 'EXCLUDED', 'EXCLUDED', 'EXCLUDED', 'EXCLUDED', 'EXCLUDED', 'EXCLUDED', 'EXCLUDED', 'EXCLUDED', 'EXCLUDED'], 'LEMMA': ['', '', '', '', '', '', '', '', '', '', ''], 'POS': ['', '', '', '', '', '', '', '', '', '', ''], 'MORPH': ['', '', '', '', '', '', '', '', '', '', ''], 'HEAD': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'DEP': ['', '', '', '', '', '', '', '', '', '', ''], 'SENT_START': [0, 0, 0, 0, 0, 0, 0, 0

In [48]:
#le process prédit du prénom pour tous les tokens
examples[0].predicted.spans

{'pollutions': [rivaill59be@clinique-lyon.fr], 'pseudo-rb': [rivaill59be@clinique-lyon.fr], 'pseudo-ml': [krier, franck rivaill59be, clinique, lyon.fr], 'ents': [], '*': [], 'ADRESSE': [krier, clinique, lyon.fr], 'TEL': [franck rivaill59be]}

In [49]:
#alors qu'il y avait 3 entités à prédire
examples[0].reference.spans

{'pseudo-ml': [krier, franck, rivaill59be@clinique-lyon.fr], 'pseudo-rb': [krier, franck, rivaill59be@clinique-lyon.fr], 'pollutions': [rivaill59be@clinique-lyon.fr]}

In [50]:
#la précision et le reacll pour la classe PRENOM est logique (un TP et 10 FP)
from edsnlp.scorers.ner import ner_exact_scorer, ner_token_scorer
token_scores = ner_token_scorer(examples,span_getter={'pseudo-ml':True})
token_scores

{'micro': {'f': 0.0,
  'p': 0.0,
  'r': 0.0,
  'tp': 0,
  'support': 11,
  'positives': 9},
 'ADRESSE': {'f': 0.0,
  'p': 0.0,
  'r': 1,
  'tp': 0,
  'support': 0,
  'positives': 5},
 'TEL': {'f': 0.0, 'p': 0.0, 'r': 1, 'tp': 0, 'support': 0, 'positives': 4},
 'NOM': {'f': 0.0, 'p': 1, 'r': 0.0, 'tp': 0, 'support': 1, 'positives': 0},
 'PRENOM': {'f': 0.0, 'p': 1, 'r': 0.0, 'tp': 0, 'support': 1, 'positives': 0},
 'MAIL': {'f': 0.0, 'p': 1, 'r': 0.0, 'tp': 0, 'support': 9, 'positives': 0}}

In [53]:
from eds_pseudo.scorer import redact_scorer
#Le redact_score est une métrique d'évaluation conçue spécifiquement pour les tâches de pseudonymisation ou d'anonymisation de texte.
#Son objectif principal n'est pas de savoir si un modèle a correctement classifié une information (ex: un NOM en tant que NOM), 
#mais de mesurer son efficacité à détecter toutes les informations sensibles qui doivent être masquées, peu importe leur catégorie.
#toutes les entités ont été détectées d'ou le bon score
redact_scores = redact_scorer(examples,span_getter=ner[1].target_span_getter)
redact_scores

{'micro': {'r': 0.8181818181818182, 'full': 0.0, 'tp': 9, 'support': 11},
 'NOM': {'r': 1, 'full': 1.0, 'tp': 1, 'support': 1},
 'PRENOM': {'r': 1, 'full': 1.0, 'tp': 1, 'support': 1},
 'MAIL': {'r': 0.7777777777777778, 'full': 0.0, 'tp': 7, 'support': 9}}

In [54]:
raw_train_docs = edsnlp.data.read_json(
    cfg['training_docs']['source']['path'],
    converter="eds.pseudo_dict2doc",
    span_setter=["pseudo-ml", "pseudo-rb", "ents"],
)
augmented_train_docs = PseudoReader(raw_train_docs,max_length=15)(nlp)
docs=list(augmented_train_docs)
doc=docs[0]
docs=[doc]
doc

2025-09-29 08:50:42.294 | INFO     | edsnlp.data.json:__init__:65 - Found 1 file under /export/home/amessager/projet/pseudo_champs_courts/src/eds_pseudo/demo/public_small_train_data.jsonl


krier franck rivaill59be@clinique-lyon.fr

In [55]:
metrics=scorer(nlp,[doc])
[m for m in metrics if  "PRENOM" in m['name']]

1it [00:00, 12.32it/s]


[{'type': 'precision',
  'name': 'Token Scores / PRENOM / Precision',
  'value': 1},
 {'type': 'recall', 'name': 'Token Scores / PRENOM / Recall', 'value': 0.0},
 {'type': 'f1', 'name': 'Token Scores / PRENOM / F1', 'value': 0.0},
 {'type': 'recall', 'name': 'Token Scores / PRENOM / Redact', 'value': 1},
 {'type': 'accuracy',
  'name': 'Token Scores / PRENOM / Redact Full',
  'value': 1.0}]

# 7 Entrainement - 1 seul étape

In [56]:
cumulated_data = defaultdict(lambda: 0.0, count=0)

iterator = itertools.chain.from_iterable(itertools.repeat(dataloader))
all_metrics = []
nlp.train(True)

<edsnlp.core.pipeline.Pipeline.train.<locals>.context at 0x7f049d9ce930>

In [57]:
batch = next(iterator)
print(len(batch))#1 seul minibatch par batch, c'est parce que le grad_accumulation token = 32000 >> 11 token par batch
mini_batch=batch[0]
mini_batch

1


{'ner': {'embedding': {'embedding': {'input_ids': FoldedTensor([[    5,  8278, 14921,    54,   379,    18,    11,   761,    30,
                    8399,  2997,  5916,    14,   586,  8688,     6]]),
    'word_offsets': FoldedTensor([ 0,  1,  2,  3,  4,  6,  7,  8,  9, 11, 12, 13]),
    'word_indices': tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
    'empty_word_indices': tensor([], dtype=torch.int64),
    'stats': {'tokens': 14, 'words': 12, 'contexts': 1},
    '__cache_key__': ('collate',
     'Transformer<139658460562256>',
     -2720329181487020680)},
   '__cache_key__': ('collate',
    'TextCnnEncoder<139657777112384>',
    -151590226722496213)},
  'stats': {'ner_words': 12},
  'targets': tensor([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 

In [58]:
ner_pipe=nlp.pipeline[4][1]#le 4ème element du pipeline
ner_pipe.forward(mini_batch['ner'])

{'loss': tensor(10.2982, grad_fn=<DivBackward0>), 'tags': None, 'probs': None}

In [59]:
ner_pipe.module_forward(mini_batch['ner'])

{'loss': tensor(9.8331, grad_fn=<DivBackward0>), 'tags': None, 'probs': None}

In [60]:
optimizer.zero_grad()
loss = torch.zeros((), device=accelerator.device)
with nlp.cache():
    for name, pipe in trained_pipes:
        output = pipe.module_forward(mini_batch[name])
        if "loss" in output:
            loss += output["loss"]
        for key, value in output.items():
            if key.endswith("loss"):
                cumulated_data[key] += float(value)
accelerator.backward(loss)

/tmp/ipykernel_2335/657222039.py:10: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:835.)
  cumulated_data[key] += float(value)


In [61]:
cumulated_data

defaultdict(<function __main__.<lambda>()>,
            {'count': 0, 'loss': 10.300644874572754})

In [62]:
mini_batch['ner']

{'embedding': {'embedding': {'input_ids': FoldedTensor([[    5,  8278, 14921,    54,   379,    18,    11,   761,    30,
                   8399,  2997,  5916,    14,   586,  8688,     6]]),
   'word_offsets': FoldedTensor([ 0,  1,  2,  3,  4,  6,  7,  8,  9, 11, 12, 13]),
   'word_indices': tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
   'empty_word_indices': tensor([], dtype=torch.int64),
   'stats': {'tokens': 14,
    'words': 12,
    'contexts': 1,
    '__batch_hash__': -4110774783818369363},
   '__cache_key__': ('collate',
    'Transformer<139658460562256>',
    -2720329181487020680),
   '__batch_hash__': 2682324099001187865},
  '__cache_key__': ('collate',
   'TextCnnEncoder<139657777112384>',
   -151590226722496213),
  '__batch_hash__': 6766140822613511373},
 'stats': {'ner_words': 12, '__batch_hash__': -8742195281669873994},
 'targets': tensor([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [63]:
ner_pipe = nlp.pipes.ner
#pour voir les tenseurs qui se propagent dans le NER
ner_pipe

TrainableNerCrf(
  (embedding): TextCnnEncoder(
    (embedding): Transformer(
      (transformer): CamembertModel(
        (embeddings): CamembertEmbeddings(
          (word_embeddings): Embedding(32007, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): CamembertEncoder(
          (layer): ModuleList(
            (0-11): 12 x CamembertLayer(
              (attention): CamembertAttention(
                (self): CamembertSdpaSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                  (dropout): Dropout(p=0.1, inplace=False)
            

In [64]:
#le modele est un transformer imbriqué dans un CNN
#ici on peut voir le résultat a travers le CNN et le transformer
ner_pipe.embedding.forward(mini_batch['ner']['embedding'])

{'embeddings': FoldedTensor([[[-1.3250, -1.2864,  0.4700,  ..., -0.2006, -0.9615, -0.5448],
                [-1.0390, -1.4065,  0.9397,  ..., -0.3261, -0.1376, -0.4719],
                [-0.4358, -1.1066,  0.9245,  ...,  0.0623, -0.4186, -0.5982],
                ...,
                [-0.5107, -0.2685,  0.8006,  ..., -0.8917, -0.1778, -0.6012],
                [-0.5157, -0.9913,  0.4111,  ..., -0.0983,  0.4137, -0.9078],
                [-0.3159, -1.1667,  0.2056,  ...,  0.3823,  0.9145, -0.9923]]],
              grad_fn=<AliasBackward0>)}

In [65]:
ner_pipe.embedding

TextCnnEncoder(
  (embedding): Transformer(
    (transformer): CamembertModel(
      (embeddings): CamembertEmbeddings(
        (word_embeddings): Embedding(32007, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): CamembertEncoder(
        (layer): ModuleList(
          (0-11): 12 x CamembertLayer(
            (attention): CamembertAttention(
              (self): CamembertSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): CamembertSelfOutput(
                (de

In [66]:
#ici on peut voir le résultat a travers juste transformer
ner_pipe.embedding.forward(mini_batch['ner']['embedding'])['embeddings']

FoldedTensor([[[-1.4553, -1.7035,  0.5198,  ..., -0.2484, -0.6932, -1.4117],
               [-0.9332, -1.1271,  0.9846,  ..., -0.2709, -0.0978, -0.3550],
               [-0.2547, -1.6417,  0.9354,  ...,  0.2114, -0.5563, -0.6580],
               ...,
               [-0.4581, -0.4772,  1.5065,  ..., -0.4194, -0.3662, -1.0208],
               [-0.6421, -0.8823,  0.8267,  ...,  0.0763, -0.3401, -1.3112],
               [-0.6204, -1.1335,  0.7036,  ...,  0.0204,  0.5909, -0.6099]]],
             grad_fn=<AliasBackward0>)

In [67]:
#a travers le CNN
result_cnn=ner_pipe.embedding.forward(mini_batch['ner']['embedding'])
#a travers le module lineaire
ner_pipe.linear.forward(result_cnn['embeddings']).shape

torch.Size([1, 12, 50])

In [68]:
#a travers le CNN
result_cnn=ner_pipe.embedding.forward(mini_batch['ner']['embedding'])
#a travers le module lineaire
result_lin=ner_pipe.linear.forward(result_cnn['embeddings'])
#a traverls le CRF BIOUL multilabel
#il faut en plus les targets et les masks, que je ne connais pas
help(ner_pipe.crf.forward)

Help on method forward in module edsnlp.pipes.trainable.layers.crf:

forward(emissions, mask, target) method of edsnlp.pipes.trainable.layers.crf.MultiLabelBIOULDecoder instance
    Compute the posterior reduced log-probabilities of the tags
    given the emissions and the transition probabilities and
    constraints of the CRF, ie the loss.


    We could use the `propagate` method but this implementation
    is faster.

    Parameters
    ----------
    emissions: torch.FloatTensor
        Shape: n_samples * n_tokens * ... * n_tags
    mask: torch.BoolTensor
        Shape: n_samples * n_tokens * ...
    target: torch.BoolTensor
        Shape: n_samples * n_tokens * ... * n_tags
        The target tags represented with 1-hot encoding
        We use 1-hot instead of long format to handle
        cases when multiple tags at a given position are
        allowed during training.

    Returns
    -------
    torch.FloatTensor
        Shape: ...
        The loss

